In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [6]:
import numpy as np
import os
import gzip

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
tf.__version__

'2.1.0'

In [7]:
# GPU设置

tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memeory_growth(gpu, True)
print(len(gpus))
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(logical_gpus))

0
0


In [ ]:
# 定义加载数据的函数，data_folder为保存gz数据的文件夹，该文件夹下有4个文件
# 'train-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte.gz',
# 't10k-labels-idx1-ubyte.gz', 't10k-images-idx3-ubyte.gz'

def load_data(data_folder):

  files = [
      'train-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte.gz',
      't10k-labels-idx1-ubyte.gz', 't10k-images-idx3-ubyte.gz'
  ]

  paths = []
  for fname in files:
    paths.append(os.path.join(data_folder,fname))

  with gzip.open(paths[0], 'rb') as lbpath:
    y_train = np.frombuffer(lbpath.read(), np.uint8, offset=8)

  with gzip.open(paths[1], 'rb') as imgpath:
    x_train = np.frombuffer(
        imgpath.read(), np.uint8, offset=16).reshape(len(y_train), 28, 28)

  with gzip.open(paths[2], 'rb') as lbpath:
    y_test = np.frombuffer(lbpath.read(), np.uint8, offset=8)

  with gzip.open(paths[3], 'rb') as imgpath:
    x_test = np.frombuffer(
        imgpath.read(), np.uint8, offset=16).reshape(len(y_test), 28, 28)

  return (x_train, y_train), (x_test, y_test)

(x_train_all, y_train_all), (x_test, y_test) = load_data('F:\\2、Deep Learning\Tensorflow\Keras_datasets\Fashion-MNIST')
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]
print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

def show_single_image(image_arr):
    plt.imshow(image_arr, cmap='binary')
    plt.show()

def show_images(n_rows, n_cols, x_data, y_data, class_names):
    assert len(x_data) == len(y_data)
    assert n_rows * n_cols < len(x_data)
    plt.figure(figsize=(n_cols * 1.4, n_rows * 1.6))
    for row in range(n_rows):
        for col in range(n_cols):
            index = n_cols * row + col
            plt.subplot(n_rows, n_cols, index + 1)
            plt.imshow(x_data[index], cmap='binary', interpolation='nearest')
            plt.axis('off')
            plt.title(class_names[y_data[index]])
    plt.show()

class_names = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
               'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']

# show_single_image(x_train[0])

# show_images(3, 5, x_train, y_train, class_names)

In [ ]:
# 对数据做归一化  
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train:[None, 28, 28] ->[None, 784] (函数需求)->[None, 28, 28]  归一化要用到除法转为float
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

In [ ]:
# 生成dataset
def make_dataset(images, labels, epochs, batch_size, shuffle = True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50)
    return dataset

batch_size = 256
# 不用repeat epochs次了，因为我们后面会在for循环中反复调用
train_dataset = make_dataset(x_train_scaled, y_train, 1, batch_size)
valid_dataset = make_dataset(x_valid_scaled, y_valid, 1, batch_size)

In [ ]:
model =keras.Sequential()
model.add(keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same',
                              activation = 'relu', input_shape = (28, 28, 1)))
model.add(keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same',
                              activation = 'relu', input_shape = (28, 28, 1)))
model.add(keras.layers.MaxPool2D(pool_size = 2))

model.add(keras.layers.Conv2D(filters = 256, kernel_size = 3, padding = 'same',
                              activation = 'relu', input_shape = (28, 28, 1)))
model.add(keras.layers.Conv2D(filters = 256, kernel_size = 3, padding = 'same',
                              activation = 'relu', input_shape = (28, 28, 1)))
model.add(keras.layers.MaxPool2D(pool_size = 2))

model.add(keras.layers.Conv2D(filters = 512, kernel_size = 3, padding = 'same',
                              activation = 'relu', input_shape = (28, 28, 1)))
model.add(keras.layers.Conv2D(filters = 512, kernel_size = 3, padding = 'same',
                              activation = 'relu', input_shape = (28, 28, 1)))
model.add(keras.layers.MaxPool2D(pool_size = 2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation = 'relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 去掉compile

In [ ]:
model.summary()

In [ ]:
# customized training loop
# 1、define loss function
# 2、define train step
# 3、define test step
# 4、for 循环完成 training loop

loss_func = keras.losses.SparseCategoricalCrossentropy(
    # 在batch_size上求和
    reduction = keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
# 计算损失的累积值
# API自带累积
test_loss = keras.metrics.Mean(name = 'test_loss')
train_accuracy = keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')
test_accuracy = keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

optimizer = keras.optimizers.SGD(lr = 0.01)

def train_step(inputs):
    images, labels = inputs
    with tf.GradientTape() as tape
        predictions = model(images, training = True)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_accuracy.update_state(labels, predictions) 
    return loss


def test_step(inputs):
    images, labels = imputs
    predictions = model(images, training = True)
    t_loss = loss_func(labels, predictions)
    # test_loss也是个累积值，可以计算从开始到现在的所有样本上损失函数的均值
    test_loss.update_state(t_loss)
    test_accuracy.update_state(labels, predictions)

    
epochs = 10 
for epoch in range(epochs):
    total_loss = 0.0
    num_batches = 0
    for x in train_dataset:
        start_time = time.time()
        total_loss += train_step(x)
        run_time = time.time() - start_time
        num_batches += 1
        print('\rtotal_loss: %3.3f, num_batches: %3.3f, average_loss: %3.3f,time: %3.3f' 
              % (total_loss, num_batches, total_loss / num_batches, run_time), end = '')
    train_loss = total_loss / num_batches
    
    # 在valid上测试一遍
    for x in valid_dataset:
        test_step(x)
    print('\rEpoch: %d, loss: %3.3f, Acc: %3.3f, Val_loss: %3.3f, Val_acc: %3.3f' 
          % (epoch + 1, train_loss, train_accuracy.result(), 
             test_loss.result(), test_accuracy.result()))
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

In [ ]:
# baseline2： 在def train_step 和def test_step 前加上@tf.dunction
# 观察运算速度

In [ ]:
# 改造为分布式

In [ ]:
# 1、生成dataset过程中添加分布式策略
def make_dataset(images, labels, epochs, batch_size, shuffle = True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50)
    return dataset
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    batch_size_per_replica = 256
    batch_size = batch_size_per_replica * len(logical_gpus)
    train_dataset = make_dataset(x_train_scaled, y_train, 1, batch_size)
    valid_dataset = make_dataset(x_valid_scaled, y_valid, 1, batch_size)
    # 改造为分布式的dataset
    # 改前：一个dataset先分成若干batch，再将batch中数据分成相应份数给对应的GPU
    # 该后：给每个GPU一个完整的dataset
    train_dataset_distribute = strategy.experimental_distribute_dataset(train_dataset)
    valid_dataset_distribute = strategy.experimental_distribute_dataset(valid_dataset)    

In [ ]:
# 2、改模型部分
with strategy.scope():
    model = keras.Sequential()
    ..
    

In [ ]:
# 3、自定义流程
with strategy.scope():
    # 因为在MirroredStrategy中，每个GPU上的数据量是 batch_size / #{gpus}
    # 如batch_size = 64， #{gpus} = 4, 每个GPU的batch_size为 16 
    # 若reduction = SUM_OVER_BATCH_SIZE，就是在16个数据上做平均，
    # 而真实情况应该是在64上测平均
    loss_func = keras.losses.SparseCategoricalCrossentropy(
        reduction = keras.losses.Reduction.NONE)
    def compute_loss(labels, predictions):
        per_replica_loss = loss_func(labels, predictions)
        # loss去除以batch_size（64），而不是batch_size / #{gpus} (16)
        # 实际上前面定义的时候 batch_size = batch_size_per_replica * #{gpus}
        return tf.nn.compute_average_loss(per_replica_loss,
                                          global_batch_size = batch_size)
    ...
    
    # 1、将train_step中的loss计算方法替换
    # 2、封装train_step和test_step，让其可以给多个GPU使用
    
    # @tf.function 后面会被封装，则这里的@tf.function就没必要了
    def train_step(inputs):
        images, labels = inputs
        with tf.GradientTape() as tape:
            predictions = model(images, training = True)
            loss = compute_loss(labels, predictions)
            ...
    
    @tf.function
    def distributed_train_step(inputs):
        # 调用train_step来获得每个GPU上的损失
        per_replica_average_loss = strategy.experimental_run_v2(train_step,
                                                                args = (inputs, ))
        # 把不同GPU上的loss聚合起来
        return strategy.reduce(tf.distribute.ReduceOp.SUM, # 聚合策略
                               per_replica_average_loss,
                               axis = None)
    ...
    
    # test_step本身不变
    @tf.function
    def distributed_train_step(inputs):
        strategy.experimental_run_v2(test_step,args = (inputs, ))
        # test不需要将loss和accuracy聚合，这是因为在test_step中本身就有一个可以累积计算的metric
        # 他们的每个GPU只要单独的调用update_state就可以了

    # 下面的代码中相应的改变调用的函数就行了